In [1]:
import random as rd
import numpy as np
import pandas as pd
import os

from scoring import Metrics
from classification import Random

In [2]:
DIRPATH = 'src'

# Import data
import_path = os.path.join(DIRPATH, 'export.npy')

data = np.load (import_path)
#data[:, :-1] = data[:, :-1]/255.  # normalise values
vals_nb = data.shape[0]

In [3]:
# Create train and test sets (19.2s in low power)

def create_sets(data, train_frac=0.05):

    # Create (train, test)
    data_shuffled = data.copy()

    np.random.seed(0)
    np.random.shuffle(data_shuffled)

    train_nb = int(vals_nb * train_frac)
    # print(f'training on {train_nb} vals')

    return data_shuffled[:train_nb], data_shuffled[train_nb:]

data_train, data_test = create_sets(data, train_frac=0.3)

In [4]:
import cv2

# Test image
image_test_path = 'src/ima_1e8ccf23-d106-4227-908c-e4fbbb1da5f8.jpg'

image_test = image_test_path.split('/')[-1]
image_test_name, _ = os.path.splitext(image_test)
id_test = image_test_name[4:]

# Import and flatten test image
image_test_path = os.path.join('src/sky-images', image_test)
image_test_array = cv2.imread (image_test_path)
image_test_shape = image_test_array.shape
image_test_array_flat = image_test_array.reshape(-1, 1, 3)# / 255.
image_test_array_flat = image_test_array_flat.squeeze()

In [5]:
from classifiers.lda import LDA
# F1 = 0.8913411645159924
# dt = 1:37min (low power)

RUN_LDA = False
if RUN_LDA:
    data_train_lda = data_train
    data_test_lda  = data_test

    # Train LDA
    lda = LDA()
    lda.fit (data_train_lda[:, :-1], data_train_lda[:, -1])

    # Test LDA
    pred_lda = lda.eval_batch(data_test_lda[:, :-1], verbose=False)
    metrics_lda = Metrics(data_test_lda[:, -1], pred_lda)
    print(metrics_lda)

    # Test on test image
    image_test_pred = lda.eval_batch(image_test_array_flat, verbose=False)
    image_test_pred = image_test_pred.reshape(image_test_shape[:2])
    cv2.imwrite(f'output/{image_test_name}-pred-lda.png', image_test_pred * 255)

In [6]:
### QDA
from classifiers.qda import QDA

# F1 = 0.8984199239563736
# dt = 6.1s (100k datapoints)

RUN_QDA = True
if RUN_QDA:
    data_train_qda = data_train
    data_test_qda  = data_test[:100000]

    # Train QDA
    qda = QDA()
    qda.fit (data_train_qda[:, :-1], data_train_qda[:, -1])

    # Test QDA
    pred_qda = qda.predict(data_test_qda[:, :-1])
    metrics_qda = Metrics(data_test_qda[:, -1], pred_qda)
    print(metrics_qda)

    # Test on test image
    image_test_pred = qda.predict(image_test_array_flat)
    image_test_pred = image_test_pred.reshape(image_test_shape[:2])
    cv2.imwrite(f'output/{image_test_name}-pred-qda.png', image_test_pred * 255)

TypeError: predict() got an unexpected keyword argument 'verbose'